In [ ]:
!pip install gradio
from google.colab import drive
import cv2
import numpy as np
import gradio as gr
from scipy.signal import medfilt
from matplotlib import pyplot as plt

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

def refine_skyline(mask):
    """
    Refines the skyline in the given binary mask by adjusting column-wise transitions.

    Parameters:
    - mask (numpy.ndarray): Binary mask to be refined.

    Returns:
    - numpy.ndarray: Refined binary mask.
    """
    # Extract the number of rows and columns from the input binary mask
    num_rows, num_cols = mask.shape

    # Iterate through each column in the binary mask
    for col_index in range(num_cols):
        current_column = mask[:, col_index]
        # Apply median filtering to the current column with a window size of 19 for smoothing
        smoothed_column = medfilt(current_column, 19)

        try:
            # Attempt to find the index of the first occurrence of 0 in the smoothed column
            first_zero_index = np.where(smoothed_column == 0)[0][0]
            # Attempt to find the index of the first occurrence of 1 in the smoothed column
            first_one_index = np.where(smoothed_column == 1)[0][0]

            # Adjust skyline if there is a significant transition
            # Check if there is a significant transition (more than 20 pixels away from the start)
            if first_zero_index > 20:
                # Adjust the skyline by setting pixels between the first occurrence of 1 and 0 to 1
                mask[first_one_index:first_zero_index, col_index] = 1
                mask[first_zero_index:, col_index] = 0
                mask[:first_one_index, col_index] = 0

        except IndexError:
            continue

    # Return the refined binary mask after adjusting column-wise transitions
    return mask

def get_sky_region_gradient(img):
    """
    Extracts the sky region using gradient-based methods.

    Parameters:
    - img (numpy.ndarray): Input image.

    Returns:
    - tuple: Tuple containing the resulting image and the refined binary mask.
    """
    height, width, _ = img.shape

    # Convert the image to grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur the image to reduce noise
    img_blurred = cv2.blur(img_gray, (9, 3))

    # Apply median blur to further reduce noise
    cv2.medianBlur(img_blurred, 5)

    # Apply Laplacian filter to enhance edges
    laplacian_output = cv2.Laplacian(img_blurred, cv2.CV_8U)

    # Create a gradient mask based on Laplacian filter output
    gradient_mask = (laplacian_output < 6).astype(np.uint8)

    # Define a morphological kernel
    morph_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))

    # Erode the gradient mask to remove small artifacts
    gradient_mask_eroded = cv2.morphologyEx(gradient_mask, cv2.MORPH_ERODE, morph_kernel)

    # Copy the original mask for further processing
    mask_copy = gradient_mask_eroded.copy()

    # Refine the mask using the refine_skyline function
    refined_mask = refine_skyline(mask_copy)

    # Apply the refined mask to the original image
    after_img = cv2.bitwise_and(img, img, mask=refined_mask)

    return after_img, refined_mask

In [7]:
def gr_sky_region_extraction(image):
    """
    Converts a Gradio Image type to a numpy array, extracts the sky region, and returns the resulting image in RGB format.

    Parameters:
    - image: Gradio Image type.

    Returns:
    - numpy.ndarray: Image in RGB format with identified sky pixels.
    """
    # Convert Gradio Image type to numpy array
    image_array = np.array(image)

    # Sky region extraction
    img_gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    masked_image, binary_mask = get_sky_region_gradient(image_array)

    # Convert the output image to RGB for Gradio compatibility
    output_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)

    return output_image_rgb

# Define the Gradio interface
demo = gr.Interface(
    fn=gr_sky_region_extraction,
    inputs="image",
    outputs="image",
    title="Sky Pixel Identification",
    description="Result： Light part indicates sky pixels, dark for non-sky",
    allow_flagging="never",
    live=True,
    )

# Launch the Gradio interface
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fb3736638c5a8fd99f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
